In [1]:
!pip install --upgrade pip setuptools wheel


In [2]:
!pip install fasttext-wheel



In [1]:
import fasttext
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
df = pd.read_csv("processed.csv") 
df.head() 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '..."


In [3]:
print(type(df['tokens'].iloc[0]))


<class 'str'>


In [4]:
with open("corpus.txt", "w", encoding="utf-8") as f:
    for tokens_str in df['tokens']:
        tokens_list = ast.literal_eval(tokens_str)
        f.write(" ".join(tokens_list) + "\n")

In [5]:
model = fasttext.train_unsupervised("corpus.txt", model="skipgram")
model.save_model("fasttext_model.bin")

In [6]:
sentence = df['tokens'].iloc[0]
print(model.get_sentence_vector(sentence))

[-0.03117303  0.12017851 -0.09793375 -0.06229379  0.02857367  0.00203843
 -0.03981128  0.01524813  0.05101157 -0.04427597 -0.08116254  0.09160688
 -0.03833751  0.0262985   0.17509778  0.01584085  0.08903384 -0.10213679
  0.06198153  0.14376116  0.03783491  0.04825466 -0.04606082 -0.01730537
  0.0032075   0.06963371 -0.08170468  0.04341748  0.03311481 -0.03887697
 -0.06392954 -0.07404617  0.06003402 -0.04234184 -0.06732631  0.01098702
  0.06020722 -0.05094877 -0.03248778 -0.06752502  0.09561636  0.11061568
  0.08284736  0.11042347  0.23013534 -0.01157455  0.06556123  0.00420542
 -0.00299908  0.06065762  0.01140256 -0.05469645  0.01091397 -0.01791675
 -0.08588101 -0.10121941 -0.00318228 -0.06189834 -0.01608436  0.08393773
  0.08856589  0.03165794  0.03737488  0.0624224  -0.02852458  0.12542897
  0.02899306  0.08025192  0.01311591 -0.06591009 -0.08331956  0.06956758
 -0.01774253 -0.09326677 -0.01572287 -0.05880982 -0.01734214 -0.05623205
  0.02423082  0.04695239 -0.0330557  -0.02050299 -0

In [9]:
def get_review_vector(tokens):
    # Join tokens into a sentence (fastText expects a string, not list)
    sentence = " ".join(tokens)
    return model.get_sentence_vector(sentence)

# Apply to DataFrame
df["unweighted_vector"] = df["tokens"].apply(get_review_vector)

df

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens,vector,unweighted_vector
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially...","[0.057335682, 0.031611666, 0.053995974, -0.005...","[0.057335682, 0.031611666, 0.053995974, -0.005..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim...","[0.044263314, 0.013135654, 0.016781278, -0.009...","[0.044263314, 0.013135654, 0.016781278, -0.009..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',...","[0.055234734, 0.021273796, 0.044271372, -0.010...","[0.055234734, 0.021273796, 0.044271372, -0.010..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'...","[0.06189355, 0.021500273, 0.046429742, -0.0006...","[0.06189355, 0.021500273, 0.046429742, -0.0006..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '...","[0.050945505, 0.032180067, 0.051167842, -0.010...","[0.050945505, 0.032180067, 0.051167842, -0.010..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19657,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses,"['happy', 'snag', 'price', 'easy', 'slip', 'cu...","[0.04632913, 0.042123564, 0.03151138, 0.002340...","[0.04632913, 0.042123564, 0.03151138, 0.002340..."
19658,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits,"['reminds', 'maternity', 'clothes', 'stretchy'...","[0.056299277, 0.01714201, 0.041920304, 0.00244...","[0.056299277, 0.01714201, 0.041920304, 0.00244..."
19659,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses,"['worked', 'glad', 'store', 'order', 'online']","[0.077798106, 0.02976948, 0.037129387, 0.00286...","[0.077798106, 0.02976948, 0.037129387, 0.00286..."
19660,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses,"['wedding', 'summer', 'medium', 'fits', 'waist...","[0.058712292, 0.017937977, 0.043994788, -0.011...","[0.058712292, 0.017937977, 0.043994788, -0.011..."


In [11]:
import ast

out_path = "unweighted_vectors.txt"

with open(out_path, "w", encoding="utf-8") as fout:
    for review_index, vec_str in enumerate(df["unweighted_vector"].values):
        # Convert string "[0.1, 0.2, ...]" into Python list
        fout.write(f"#{review_index}," + ",".join(map(str, vec_str)) + "\n")

print(f"[OK] Wrote {len(df)} lines to {out_path}")

# Preview first 3 lines
with open(out_path, "r", encoding="utf-8") as f:
    for _ in range(3):
        print(f.readline().rstrip())

[OK] Wrote 19662 lines to unweighted_vectors.txt
#0,0.057335682,0.031611666,0.053995974,-0.0050533316,0.0064341384,0.026344934,0.015404279,0.012648252,0.020666525,0.0061200433,0.012353387,0.037816696,-0.016656496,0.007186979,0.08338787,0.017531987,0.010452897,0.03967139,-0.019307125,0.016212959,-0.0361864,-0.025356865,-0.024756629,0.053909834,-0.044659,0.016474927,-0.0054521044,0.0019241977,0.034850195,-0.042122614,0.008781958,0.07441728,0.0006349746,-0.020025134,0.027674017,0.0034460258,-0.011675759,0.0022057486,0.00021276204,-0.002337548,0.034565978,-0.030454168,0.03446253,-0.025398064,-0.0047816816,0.06273663,-0.053863995,-0.05182905,0.040984724,-0.03184524,-0.020510701,-0.017372929,0.05476249,0.0032794103,0.02147852,0.022472842,0.018112512,0.008426445,-0.052696835,-0.08992932,0.044079706,0.033847034,0.0030675386,0.013996939,-0.002594789,0.03221885,0.051251315,0.0066677523,-0.055868804,0.029947437,-0.011920773,-0.025083045,-0.0012819612,0.00088091847,0.0208225,0.004570246,0.03617696

In [ ]:
# load corpus into sentences for vectorizer
with open("corpus.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f]

# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

In [21]:
# Map token indices to their corresponding tokens
index_to_token = {j: t for t, j in tfidf_vectorizer.vocabulary_.items()}

# Get the embedding dimension from the model
embedding_dimension = model.get_dimension()

# Create a weight matrix for the embeddings
weight_matrix = np.zeros((tfidf_matrix.shape[0], embedding_dimension))

for i in range(tfidf_matrix.shape[0]):
    row = tfidf_matrix.getrow(i)
    if row.nnz == 0:
        continue

    vector_sum = np.zeros(embedding_dimension, dtype=np.float32)
    weight_sum = 0.0

    # Compute the weighted sum of word vectors
    for j, w in zip(row.indices, row.data):
        token = index_to_token[j]
        vec = model.get_word_vector(token)
        vector_sum += vec * float(w)
        weight_sum += float(w)

    # Normalize the vector by the weight sum
    if weight_sum > 0:
        weight_matrix[i] = vector_sum / weight_sum



In [ ]:
df["weighted_vector"] = [vec.tolist() for vec in weight_matrix]

df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens,vector,weighted_embedding
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially...","[0.058459032, 0.028506024, 0.0549225, -0.00385...","[-0.11857645213603973, 0.284871906042099, -0.0..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim...","[0.044263314, 0.013135654, 0.016781278, -0.009...","[-0.15346971154212952, 0.2994864284992218, -0...."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',...","[0.058937944, 0.011035831, 0.047325734, -0.006...","[-0.06939209252595901, 0.23377594351768494, -0..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'...","[0.06352168, 0.01699908, 0.047772598, 0.001047...","[-0.08697155863046646, 0.23875336349010468, -0..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '...","[0.052672256, 0.027406245, 0.052592043, -0.009...","[-0.1943133920431137, 0.248451367020607, 0.053..."


In [ ]:
import ast

out_path = "weighted_vectors.txt"

with open(out_path, "w", encoding="utf-8") as fout:
    for review_index, vec_str in enumerate(df["weighted_vector"].values):
        # Convert string "[0.1, 0.2, ...]" into Python list
        fout.write(f"#{review_index}," + ",".join(map(str, vec_str)) + "\n")

print(f"[OK] Wrote {len(df)} lines to {out_path}")

# Preview first 3 lines
with open(out_path, "r", encoding="utf-8") as f:
    for _ in range(3):
        print(f.readline().rstrip())